In [ ]:
# Heres where I upload the data to a dataframe and do a lot of imports
! pip install unidecode
! pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from collections import Counter
import unidecode

class PickupLinesDataset(Dataset):
  def __init__(self, data, chunk_len):
    self.data = data
    self.chunk_len = chunk_len
    self.chars = tuple(set(data))
    self.char_to_int = {ch: i for i, ch in enumerate(self.chars)}
    self.int_to_char = dict(enumerate(self.chars))

  def __len__(self):
    return len(self.data) - self.chunk_len

  def __getitem__(self, index):
    return (
        torch.tensor([self.char_to_int[ch] for ch in self.data[index:index+self.chunk_len]]),
        torch.tensor([self.char_to_int[self.data[index+self.chunk_len]]])
    )


url = 'https://raw.githubusercontent.com/jacobjones36/pickuplines/main/lines_revised.csv?token=GHSAT0AAAAAACMV2YC3Z7ARNOE3BZ3NWVYYZRHAEDQ'
df = pd.read_csv(url)
#print(df['Normal'][1000])
normal = df['Normal'].astype(str).str.strip().replace('nan', pd.NA).dropna().tolist()
knock = df['Knock'].astype(str).str.strip().replace('nan', pd.NA).dropna().tolist()
math = df['Math'].astype(str).str.strip().replace('nan', pd.NA).dropna().tolist()
science = df['Science'].astype(str).str.strip().replace('nan', pd.NA).dropna().tolist()
sleep = df['Sleep'].astype(str).str.strip().replace('nan', pd.NA).dropna().tolist()
church = df['Church'].astype(str).str.strip().replace('nan', pd.NA).dropna().tolist()
#math = '\n'.join(math)
#math = unidecode.unidecode(math)
#data = PickupLinesDataset(math, chunk_len=100)
#dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [ ]:
class PickupLinesCategoryDataset(Dataset):
    def __init__(self, categories, chunk_len):
        self.chunk_len = chunk_len
        self.data = []
        self.vocab = set()

        # Initialize category mappings
        self.categories = list(categories.keys())
        self.cat_to_int = {cat: i for i, cat in enumerate(self.categories)}

        # Build vocabulary and prepare data
        for category, lines in categories.items():
            for line in lines:
                self.vocab.update(line)
                if len(line) > self.chunk_len:
                    for start_idx in range(len(line) - self.chunk_len):
                        end_idx = start_idx + self.chunk_len + 1
                        input_seq = line[start_idx:end_idx]
                        self.data.append((category, input_seq))

        self.chars = sorted(list(self.vocab))
        self.char_to_int = {ch: i for i, ch in enumerate(self.chars)}
        self.int_to_char = {i: ch for i, ch in enumerate(self.chars)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        category, sequence = self.data[index]
        cat_idx = self.cat_to_int[category]
        input_seq = torch.tensor([self.char_to_int[ch] for ch in sequence[:-1]], dtype=torch.long)
        target_seq = torch.tensor([self.char_to_int[ch] for ch in sequence[1:]], dtype=torch.long)
        return cat_idx, input_seq, target_seq




In [ ]:
class PickupLineCategoryGenerator(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, category_count, category_embedding_dim):
        super(PickupLineCategoryGenerator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.category_embedding = nn.Embedding(category_count, category_embedding_dim)
        self.lstm = nn.LSTM(embedding_dim + category_embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, category, seq, hidden=None):
        embedded_seq = self.embedding(seq)
        embedded_cat = self.category_embedding(category).unsqueeze(1).expand(-1, seq.size(1), -1)
        combined_embed = torch.cat([embedded_seq, embedded_cat], dim=2)
        output, hidden = self.lstm(combined_embed, hidden)
        output = self.fc(output)
        return output, hidden


In [ ]:
def train(model, dataloader, epochs, optimizer, criterion, scheduler, device):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for cat_idx, input_seq, target_seq in dataloader:
            cat_idx = cat_idx.to(device)
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            optimizer.zero_grad()
            output, _ = model(cat_idx, input_seq)
            output_flat = output.view(-1, output.size(-1))
            loss = criterion(output_flat, target_seq.view(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        scheduler.step()
        print(f'Epoch {epoch+1}: Loss = {total_loss / len(dataloader)}')

# Assuming 'device' and 'model' are defined and properly initialized
# Assuming 'dataloader' is an instance of DataLoader using PickupLinesDataset


In [ ]:
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
categories = {
    'normal': normal,
    'math': math,
    'science': science,
    'church': church,
    'sleep': sleep,
    'knock': knock
}

dataset = PickupLinesCategoryDataset(categories, chunk_len=10)
dataloaderC = DataLoader(dataset, batch_size=128, shuffle=True)

vocab_size = len(dataset.vocab)
category_count = len(categories)
embedding_dim = 256
hidden_dim = 512
category_embedding_dim = 50

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
modelC = PickupLineCategoryGenerator(vocab_size, embedding_dim, hidden_dim, category_count, category_embedding_dim)
modelC.to(device)
optimizerC = torch.optim.Adam(modelC.parameters(), lr=0.001)
criterionC = nn.CrossEntropyLoss()
schedulerC = StepLR(optimizerC, step_size=30, gamma=0.1)
trainCategory(modelC, dataloaderC, 100, optimizerC, criterionC, schedulerC, device)

Epoch 1, Loss: 1.6558877647358134
Epoch 2, Loss: 1.2721155292672086
Epoch 3, Loss: 1.1291076218373335
Epoch 4, Loss: 1.0412259604684753
Epoch 5, Loss: 0.9816602350400063
Epoch 6, Loss: 0.9402606706052428
Epoch 7, Loss: 0.910374926327416
Epoch 8, Loss: 0.8880857254715483
Epoch 9, Loss: 0.8712996635993902
Epoch 10, Loss: 0.8579392809813164
Epoch 11, Loss: 0.8473520679195432
Epoch 12, Loss: 0.8381845768972284
Epoch 13, Loss: 0.8309573504425065
Epoch 14, Loss: 0.8242782054156282
Epoch 15, Loss: 0.8193558243193642
Epoch 16, Loss: 0.8142164821918118
Epoch 17, Loss: 0.8099778249813195
Epoch 18, Loss: 0.8064659871821359
Epoch 19, Loss: 0.8032053386456527
Epoch 20, Loss: 0.7998407938060224
Epoch 21, Loss: 0.7970773475137816
Epoch 22, Loss: 0.7943911250366036
Epoch 23, Loss: 0.7921434683148381
Epoch 24, Loss: 0.7904982312628077
Epoch 25, Loss: 0.7881974586605156
Epoch 26, Loss: 0.7866294978184546
Epoch 27, Loss: 0.7848938127379472
Epoch 28, Loss: 0.783250671233574
Epoch 29, Loss: 0.7813546456808

In [ ]:
def generate(model, dataset, category, start_phrase, max_length):
    model.eval()
    input_idx = [dataset.char_to_int[char] for char in start_phrase]
    input_tensor = torch.tensor([input_idx], dtype=torch.long).to(device)
    cat_idx = torch.tensor([dataset.cat_to_int[category]], dtype=torch.long).to(device)

    output_text = start_phrase
    hidden = None

    with torch.no_grad():
        for _ in range(max_length):
            output, hidden = model(cat_idx, input_tensor, hidden)
            probabilities = torch.softmax(output[:, -1, :], dim=-1)
            char_idx = torch.multinomial(probabilities, 1).item()
            next_char = dataset.int_to_char[char_idx]
            output_text += next_char
            input_tensor = torch.tensor([[char_idx]], dtype=torch.long).to(device)
            if next_char == '\n':
                break

    return output_text
print(generate(modelC, dataset, 'math', 'You', 150))

NameError: name 'modelC' is not defined

In [ ]:
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
def repackage_hidden(h):
  """Wraps hidden states in new Tensors, to detach them from their history."""
  if isinstance(h, torch.Tensor):
    return h.detach()
  else:
    return tuple(repackage_hidden(v) for v in h)

def train(model, dataloader, epochs, optimizer, criterion, scheduler, device):
  model.train()
  for epoch in range(epochs):

    #running_loss = 0.0
    #hidden = model.init_hidden(dataloader.batch_size)
    total_loss = 0
    for inputs, targets in dataloader:
      inputs, targets = inputs.to(device), targets.to(device)
      optimizer.zero_grad()
      batch_size = inputs.size(0)
      hidden = model.init_hidden(batch_size)

      output, hidden = model(inputs, hidden)
      loss = criterion(output, targets.view(-1))
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
      # running_loss += loss.item() * inputs.size(0)

    print(f'Epoch {epoch+1}, Avg Loss: {total_loss / len(dataloader)}')
    scheduler.step()
    # print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader.dataset)}')
    # validate_model(model, valid_loader, criterion, device)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PickupLineGenerator(len(dataset.chars), 256, 512, 2)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
# Alternatively, use ReduceLROnPlateau for more dynamic scheduling
# scheduler = ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.1, verbose=True)
train(model, dataloader, 25, optimizer, criterion, scheduler, device)

Epoch 1, Avg Loss: 2.478307977089515
Epoch 2, Avg Loss: 1.585894327897292
Epoch 3, Avg Loss: 1.36457934654676
Epoch 4, Avg Loss: 1.2004481013004595
Epoch 5, Avg Loss: 1.0720143813353318
Epoch 6, Avg Loss: 0.9708557220605704
Epoch 7, Avg Loss: 0.8826700494839594
Epoch 8, Avg Loss: 0.7977023619871874
Epoch 9, Avg Loss: 0.7329606890678406
Epoch 10, Avg Loss: 0.6596093980165628
Epoch 11, Avg Loss: 0.6053705050395085
Epoch 12, Avg Loss: 0.5573770330502437
Epoch 13, Avg Loss: 0.5118330996770125
Epoch 14, Avg Loss: 0.46768747659829946
Epoch 15, Avg Loss: 0.43860442088200496
Epoch 16, Avg Loss: 0.4063060992039167
Epoch 17, Avg Loss: 0.3541602473992568
Epoch 18, Avg Loss: 0.3367928030399176
Epoch 19, Avg Loss: 0.31293208576165715
Epoch 20, Avg Loss: 0.280379421206621
Epoch 21, Avg Loss: 0.27380289916808787
Epoch 22, Avg Loss: 0.24839233595591326
Epoch 23, Avg Loss: 0.23080607956418625
Epoch 24, Avg Loss: 0.21970850309500328
Epoch 25, Avg Loss: 0.2166504483956557


In [ ]:
def generate_categorical_pickup_line(model, category, start_phrase, max_length):
    model.eval()  # Set the model to evaluation mode
    device = next(model.parameters()).device  # Ensure the tensor is on the same device as the model

    # Convert start_phrase to tensor
    input_seq = torch.tensor([[dataset.char_to_int[ch] for ch in start_phrase]], dtype=torch.long).to(device)
    category_tensor = torch.tensor([dataset.cat_to_int[category]], dtype=torch.long).to(device)

    output_line = start_phrase

    hidden = None  # Initialize hidden state (if your model uses one and needs manual initialization)

    with torch.no_grad():
        for _ in range(max_length):
            output, hidden = model(category_tensor, input_seq, hidden)
            # Take the last time step from the last layer of the LSTM output
            probabilities = torch.softmax(output[0, -1], dim=-1)
            char_idx = torch.multinomial(probabilities, 1).item()
            next_char = dataset.int_to_char[char_idx]  # Correct dictionary for character lookup

            output_line += next_char
            # Prepare the next input to be the character just generated
            input_seq = torch.tensor([[char_idx]], dtype=torch.long).to(device)

            if next_char == '\n':  # Optionally: stop if the model generates a newline character
                break

    return output_line


print(generate_categorical_pickup_line(modelC, 'normal', 'You must be ', 50))


AttributeError: 'PickupLinesCategoryDataset' object has no attribute 'int_to_char'

In [ ]:
class PickupLineGenerator(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
    super(PickupLineGenerator, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=1, batch_first=True)
    self.dropout = nn.Dropout(dropout)
    self.lstm1 = nn.LSTM(hidden_dim, hidden_dim, num_layers=1, batch_first=True)
    self.dropout1 = nn.Dropout(dropout)
    #self.layer_norm = nn.LayerNorm(hidden_dim)
    self.fc = nn.Linear(hidden_dim, vocab_size)

  def forward(self, x, hidden):
    embedded = self.embedding(x)
    output, hidden0 = self.lstm(embedded, hidden[0])
    output = self.dropout(output)
    output, hidden1 = self.lstm1(output, hidden[1])
    output = self.dropout1(output)
    #output = self.layer_norm(output[:, -1, :])
    output = self.fc(output[:, -1, :])
    return output, (hidden0, hidden1)

  def init_hidden(self, batch_size):
    weight = next(self.parameters()).data
    #hidden = (weight.new(self.lstm.num_layers, batch_size, self.lstm.hidden_size).zero_(),
             #weight.new(self.lstm.num_layers, batch_size, self.lstm.hidden_size).zero_())
    hidden = (
            (weight.new_zeros(1, batch_size, self.lstm.hidden_size),
             weight.new_zeros(1, batch_size, self.lstm.hidden_size)),
            (weight.new_zeros(1, batch_size, self.lstm1.hidden_size),
             weight.new_zeros(1, batch_size, self.lstm1.hidden_size))
        )
    return hidden

In [ ]:
import torch.nn.functional as F
def generate(model, start_str, len_gen, temperature=1.0):
    model.eval()
    device = next(model.parameters()).device

    hidden = model.init_hidden(1)
    input_seq = torch.tensor([[dataset.char_to_int[ch] for ch in start_str]], dtype=torch.long).to(device)
    predicted = start_str

    for _ in range(len_gen):
        output, hidden = model(input_seq, hidden)
        output_dist = F.softmax(output.data.view(-1) / temperature, dim=0)
        # output_dist = output.data.view(-1).div(temperature).exp()
        top_char = torch.multinomial(output_dist, 1)[0]
        predicted_char = dataset.int_to_char[top_char.item()]
        predicted += predicted_char
        input_seq = torch.tensor([[top_char]], dtype=torch.long).to(device)

    return predicted


# Generate a new pickup line
print(generate(model, 'Knock', 100))

Knock Knock Who's there? Mirra. Mirra who? Howard you like a big kiss?
Knock Knock Who's there? Mistleter


In [ ]:
# We getting all the characters here
! pip install unidecode
! pip install torch

import unidecode
import string
import random

additional_characters = ['’', '…', '—', '“', '”']
all_characters = string.printable + ''.join(additional_characters)
n_characters = len(all_characters)

In [ ]:
import torch
import torch.nn.functional as F

chunk_len = 100
X = []
Y = []
for i in range(0, n_chars - chunk_len, 1):
  inp = all_lines[i:i+chunk_len]
  out = all_lines[i+chunk_len]
  X.append([chars_to_int[char] for char in inp])
  Y.append([chars_to_int[out]])
n_patterns = len(X)

X = np.reshape(X, (n_patterns, chunk_len, 1))
X = X/float(n_vocab)
Y = F.one_hot(torch.tensor(Y))


tensor([[[0, 1, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]]])


In [ ]:
print(Y.size())
print()

torch.Size([414317, 1, 82])


In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(pickup_lines), specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])  # default index for unknown words

# Numericalizing text
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]

# Convert texts to padded sequence of indices
max_length = min(len(text_pipeline(line)) for line in pickup_lines)
print(max_length)
padded_sequences = [text_pipeline(line) + [vocab['<pad>']] * (max_length - len(text_pipeline(line))) for line in pickup_lines]

# Convert to tensor
import torch
prepped_data = torch.tensor(padded_sequences, dtype=torch.long)
input_seq = prepped_data[:, :-1]
target_seq = prepped_data[:, 1:]

In [ ]:
# Here well get a random chunk
import random

chunk_len = 200


def random_chunk():
  chunk = ''
  while len(chunk) < chunk_len:
    i = random.randint(0, df.size)
    if len(pickup_lines[i]) + len(chunk) < chunk_len:
      chunk += pickup_lines[i]
      i += 1
    else:
      remaining = chunk_len - len(chunk)
      chunk += pickup_lines[i][:remaining]
  return chunk

print(random_chunk())

Baby you thicker than a novel, and I wanna read all yo pagesIf looks could kill you would be a weapon of mass destruction.Here, let me chalk up those jugs for you.Are you the sun? Because you're the c


In [ ]:
# Idk what the fuck this does but its dope
import torch
char_to_index = {ch: idx for idx, ch in enumerate(all_characters)}
# Turn string into list of longs
def char_tensor(string):
  tensor = torch.zeros(len(string), dytpe=torch.long)
  for c in range(len(string)):
    tensor[c] = char_to_index.get(string[c], 0)

  return tensor

print(char_tensor('abcDEF'))

tensor([10, 11, 12, 39, 40, 41])


In [ ]:
# Some more shit that's dope
def random_training_set():
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

In [ ]:
# NICE an RNN
import torch
import torch.nn as nn
import torch.nn.functional as F
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers=1):
    super(RNN, self).__init__()
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.n_layers = n_layers
    self.hidden_size = hidden_size
    self.relu = nn.ReLU()
    self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
    self.fc = nn.Linear(hidden_size, output_size)
    self.output_size = output_size

  def forward(self, input, hidden):
    embedded = self.embedding(input).view(1, 1, -1)
    output, hidden = self.gru(embedded, hidden)
    output = self.fc(output)
    out_decoded = self.relu(output)

    return out_decoded, hidden

  def init_hidden(self):
    return torch.zeros(self.n_layers, 1, self.hidden_size)

In [ ]:
# Train this shit
# NOTE: decoder_optimizer, decoder, and criterion will be defined below as global variables
def train(inp, target):
  decoder_optimizer.zero_grad()
  loss = 0
  hidden = decoder.init_hidden()
  for i, input in enumerate(inp):
    output, hidden = decoder(input, hidden)
    loss += criterion(output.squeeze(0), target[i].unsqueeze(0))
  loss.backward()
  decoder_optimizer.step()

  return loss.item()
  """criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
  inp = inp.unsqueeze(1)
  for epoch in range(num_epochs):
    total_loss = 0
    model.train()
    for i in range (0, inp.size(0), batch_size):
      hidden = model.init_hidden()
      optimizer.zero_grad()
      inputs = inp[i:i+batch_size]
      targets = target[i:i+batch_size]

      outputs, hidden = model(inputs, hidden)
      loss = criterion(outputs.view(-1, model.output_size), targets.view(-1))
      loss.backward()
      optimizer.step()
      total_loss += loss.item()

    print('Epoch: {}/{}.............'.format(epoch+1, num_epochs), end=' ')
    print("Loss: {:.4f}".format(total_loss / inp.size(0)))"""



In [ ]:
# Sample output fuck shit
def sample_outputs(output, temperature):
    """Takes in a vector of unnormalized probability weights and samples a character from the distribution"""
    # As temperature approaches 0, this sampling function becomes argmax (no randomness)
    # As temperature approaches infinity, this sampling function becomes a purely random choice
    return torch.multinomial(torch.exp(output / temperature), 1)

def evaluate(prime_str='A', predict_len=100, temperature=0.8):
  ## initialize hidden state, initialize other useful variables
    # your code here
  ## /
  hidden_state = decoder.init_hidden()
  input = char_tensor(prime_str)[-1]
  generated_text = [prime_str]
  with torch.no_grad():
    for i in range(predict_len):
      output, hidden_state = decoder(input, hidden_state)
      sample_output = sample_outputs(output.squeeze(0), temperature)
      next_char = all_characters[sample_output.item()]
      generated_text.append(next_char)
      input = char_tensor(next_char)


  return ''.join(generated_text)



In [ ]:
# Set up the model bitch
import time
n_epochs = 2000
print_every = 200
plot_every = 10
hidden_size = 200
n_layers = 3
lr = 0.001
decoder = nn.Sequential()
decoder.Add(nn.LTSM(256, input_shape=()))
#decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
#train(model, input_seq, target_seq, n_epochs)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
start = time.time()
all_losses = []
loss_avg = 0

In [ ]:
# Run this shit
# n_epochs = 2000
for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set())
  loss_avg += loss_

  if epoch % print_every == 0:
      print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
      print(evaluate('Wh', 100), '\n')

  if epoch % plot_every == 0:
      all_losses.append(loss_avg / plot_every)
      loss_avg = 0

[120.60692834854126 (200 10%) 520.1180]
Wh/Pn.. /ror thire a f9eDNas a hout I anghrarnase got me got od the beat \an mauslet in] nouls a a but 

[234.40011167526245 (400 20%) 579.9518]
Wh!'-s hot are the tonight 	o deing a I don lood ond ithing to here of mabe# a jide fise …eer.I me th 

[367.22923731803894 (600 30%) 433.1595]
Wh?9’“\5-9QFu= meive a the siving in intiis a dindart, let is I beaut from hought.Ca* RoTnoU mapZ mab 

[482.3747613430023 (800 40%) 464.2769]
Wh+^re the get ^rath it and ~ater some.Fore his tose 791 Gom.,I donejVber sart sas more Me , that dene 

[594.611447095871 (1000 50%) 484.6093]
Wh\…Q*Trite $(7{ for line <oz of the hould life.I PW—'le the see isquint out is of to mineges.7Ro! Qn 

[707.1234545707703 (1200 60%) 403.3609]
Wh.[what 
omestest …as reall ammar—ing shat is the name Qreall! % ?Can ?I or 6a[ the onl6 me into go a 

[818.8365364074707 (1400 70%) 334.5836]
Wher some an an outruate in ul”n our not a smaling out out to see ous for some onl, uss lo

In [ ]:
for i in range(10):
  start_strings = [" Th", " wh", " he", " I ", " ca", " G", " lo", " ra"]
  start = random.randint(0,len(start_strings)-1)
  print(start_strings[start])
#   all_characters.index(string[c])
  print(evaluate(start_strings[start], 200), '\n')


In [ ]:
def generate_line(model, start_phrase="Hello", generation_length=75):
  model.eval()
  words = start_phrase.split()
  state = model.init_hidden()
  for w in words[:-1]:
    idx = torch.tensor([[vocab[w]]], dtype=torch.long)
    output, state = model(idx, state)
  new_words = []
  next_word = words[-1]
  for _ in range(generation_length):
    idx = torch.tensor([[vocab[next_word]]], dtype=torch.long)
    output, state = model(idx, state)
    next_word = output.squeeze(0).squeeze(0).argmax().item()
    next_word = vocab.lookup_token(next_word)
    new_words.append(next_word)
  return ' '.join(words + new_words)

print(generate_line(model, start_phrase="Are you a", generation_length=75))

NameError: name 'model' is not defined